# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

Basic sanity-check notebook for Drop-In Knowledge Mode runtime.

Goals:
  - Verify that DropInKnowledgeRuntime can:
      * create or load a session,
      * append user and assistant messages,
      * build conversation history from SessionStore,
      * return a RuntimeAnswer object.
  - Use InMemorySessionStore for simplicity.
  - Keep LLM integration as placeholder for now.

Create config, session store, runtime

In [3]:
from datetime import datetime
from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.llm_adapters.base import LLMAdapterRegistry, LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

# Instantiate the in-memory session store
session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

# Instantiate a LLM adapter
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

TENANT = "intergrax"
CORPUS = "intergrax-strategy"
VERSION = "v1"

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    llm_label="llm-adapter",
    enable_rag=False,        # disabled for this initial sanity check
    enable_websearch=False,  # disabled for this initial sanity check
    tools_mode="off",      # disabled for this initial sanity check
)

runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
)

print("Runtime initialized at", datetime.utcnow().isoformat())

Runtime initialized at 2025-12-18T17:50:46.346770


C:\Users\XPS\AppData\Local\Temp\ipykernel_3028\2920471571.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Runtime initialized at", datetime.utcnow().isoformat())


First request: create a new session

In [5]:
# Define user and session identifiers
import pprint
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


user_id = "demo-user-001"
session_id = "demo-session-001"

request_1 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="Hello Intergrax runtime, this is my first message.",
    metadata={"source": "notebook-demo"},
)

answer_1 = await runtime.run(request_1)

print("=== ANSWER 1 ===")
print(answer_1.answer)
print("\nRoute info:")
print(answer_1.route)
print("\nStats:")
print(answer_1.stats)
print("\nDebug trace:")
print(answer_1.debug_trace)


=== ANSWER 1 ===
It looks like we've got a repeat of your initial message!

Just to confirm, I'm not actually aware of the Integrax Runtime being a real platform or service. It's possible that it's a custom or internal system that you're familiar with.

If you'd like to continue exploring the Integrax Runtime, could you please provide more context about what it is and how it works? I'll do my best to help you navigate its features and capabilities.

Route info:
RouteInfo(used_rag=False, used_websearch=False, used_tools=False, used_user_profile=False, used_user_longterm_memory=False, strategy='llm_only', extra={})

Stats:
RuntimeStats(total_tokens=None, input_tokens=None, output_tokens=None, rag_tokens=None, websearch_tokens=None, tool_tokens=None, duration_ms=None, extra={})

Debug trace:
{'session_id': 'demo-session-001', 'user_id': 'demo-user-001', 'config': {'llm_label': 'llm-adapter', 'embedding_label': 'default-embedding', 'vectorstore_label': 'default-vectorstore'}, 'memory_layer

Inspect session state after first request

In [6]:
session = await session_manager.get_session(answer_1.debug_trace["session_id"])
messages = await session_manager.get_history(session_id=answer_1.debug_trace["session_id"])

print("=== SESSION AFTER FIRST REQUEST ===")
print("Session ID:", session.id)
print("User ID:", session.user_id)
print("Messages count:", len(messages))
print("Created at:", session.created_at)
print("Updated at:", session.updated_at)

print("\nMessages:")
for idx, msg in enumerate(messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== SESSION AFTER FIRST REQUEST ===
Session ID: demo-session-001
User ID: demo-user-001
Messages count: 4
Created at: 2025-12-18 17:50:49.163095+00:00
Updated at: 2025-12-18 17:51:12.946537+00:00

Messages:
  [1] role='user', created_at=2025-12-18T17:50:49.163095+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-12-18T17:50:56.537793+00:00
      content="Welcome to the Integrax Runtime! This is an exciting platform for integrating various services and applications.\n\nAs it appears that I'm not aware of your specific setup or environment, could you please provide more context about what you're trying to achieve with this platform? What kind of integrations are you planning, or what's the purpose behind using the Integrax Runtime in your project?\n\nI'm here to help and assist you throughout your integration journey."
  [3] role='user', created_at=2025-12-18T17:51:10.513592+00:00
      content='Hello Intergrax runtime, this 

Second request: same session, history should grow

In [8]:
request_2 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="This is my second message in the same session.",
    metadata={"source": "notebook-demo"},
)

answer_2 = await runtime.run(request_2)

print("=== ANSWER 2 ===")
print(answer_2.answer)
print("\nRoute info:")
print(answer_2.route)
print("\nDebug trace:")
print(answer_2.debug_trace)

session_2 = await session_manager.get_session(answer_2.debug_trace["session_id"])
messages = await session_manager.get_history(session_id=answer_1.debug_trace["session_id"])

print("\n=== SESSION AFTER SECOND REQUEST ===")
print("Session ID:", session_2.id)
print("Messages count:", len(messages))
for idx, msg in enumerate(messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== ANSWER 2 ===
It looks like you've sent another duplicate message!

Just to confirm, our conversation has just started, and we haven't discussed anything yet. If you'd like to start fresh or ask a question about the Integrax Runtime (which I still don't have any information on), feel free to type something new.

I'm here to help and provide assistance when needed!

Route info:
RouteInfo(used_rag=False, used_websearch=False, used_tools=False, used_user_profile=False, used_user_longterm_memory=False, strategy='llm_only', extra={})

Debug trace:
{'session_id': 'demo-session-001', 'user_id': 'demo-user-001', 'config': {'llm_label': 'llm-adapter', 'embedding_label': 'default-embedding', 'vectorstore_label': 'default-vectorstore'}, 'memory_layer': {'implemented': True, 'has_user_profile_instructions': False, 'has_org_profile_instructions': False, 'enable_user_profile_memory': True, 'enable_org_profile_memory': True}, 'steps': [{'timestamp': '2025-12-18T17:51:33.841745+00:00', 'component':

Verify that history length is limited by config

In [ ]:
from intergrax.llm.messages import ChatMessage

# For debugging: call the history builder directly
session_for_history = await session_manager.get_session(answer_2.debug_trace["session_id"])
history = await runtime._history_layer._build_chat_history(session_for_history)

print("=== BUILT CHAT HISTORY ===")
print("History length:", len(history))
for idx, msg in enumerate(history, start=1):
    assert isinstance(msg, ChatMessage)
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== BUILT CHAT HISTORY ===
History length: 4
  [1] role='user', created_at=2025-11-21T11:24:54.601460+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-11-21T11:24:59.433174+00:00
      content="Welcome to our Intergrax runtime environment.\n\nWhat brings you here today? Do you have any specific tasks or projects in mind that you'd like to work on, or would you like some suggestions on how to get started with the Intergrax platform?"
  [3] role='user', created_at=2025-11-21T11:24:59.446306+00:00
      content='This is my second message in the same session.'
  [4] role='assistant', created_at=2025-11-21T11:25:01.244703+00:00
      content="So we're still going strong after your first message! Just for clarity, I should mention that this conversation just started, and there's no prior context or messages to recall from a previous session. But feel free to start fresh and ask me anything you'd like to know about Intergrax runt